In [1]:
import pandas as pd
import numpy as np
import netCDF4 as nc
import numpy as np
import os
from datetime import datetime

In [2]:
def extract_point(gridded_product, df_points, var_name):
    st_names = df_points.iloc[:,0].values
    coords = df_points.iloc[:,1:3].values.tolist()
    
   
    start = datetime.strptime("1981-01-01", "%Y-%m-%d")
    end = datetime.strptime("2020-12-31", "%Y-%m-%d")
    datelist = pd.date_range(start,end)
    years = [str(year) for year in range(start.year, end.year +1)]

    if gridded_product == "chirps_daily" and var_name == 'precip':
        path = "Z:\\1.Data\\Honduras\\Clima\\Productos grillados\\CHIRPS\\daily"
        files_1 = os.listdir(path)
        files_2 = [files_1[i] for i in range(len(files_1)) if any(s in files_1[i] for s in years)]
        files = [os.path.join(path, f) for f in files_2 if f.endswith('.nc4')]

    if gridded_product == "chirps_monthly" and var_name == 'precip':
        path = "Z:\\1.Data\\Honduras\\Clima\\Productos grillados\\CHIRPS\\monthly"
        files_1 = os.listdir(path)
        files_2 = [files_1[i] for i in range(len(files_1)) if any(s in files_1[i] for s in years)]
        files = [os.path.join(path, f) for f in files_2 if f.endswith('.nc4')]

    if gridded_product == "agera5" and var_name == 'precip':
        path = "Z:\\1.Data\\Honduras\\Clima\\Productos grillados\\AgERA5"
        files_1 = os.listdir(path)
        files_2 = [files_1[i] for i in range(len(files_1)) if any(s in files_1[i] for s in years)]
        files = [os.path.join(path, f) for f in files_2 if f.endswith('.nc4') and 'prec' in f]


    if gridded_product == "agera5" and var_name == 'tmax':
        path = "Z:\\1.Data\\Honduras\\Clima\\Productos grillados\\AgERA5"
        files_1 = os.listdir(path)
        files_2 = [files_1[i] for i in range(len(files_1)) if any(s in files_1[i] for s in years)]
        files = [os.path.join(path, f) for f in files_2 if f.endswith('.nc4') and 'tmax' in f]

    if gridded_product == "agera5" and var_name == 'tmin':
        path = "Z:\\1.Data\\Honduras\\Clima\\Productos grillados\\AgERA5"
        files_1 = os.listdir(path)
        files_2 = [files_1[i] for i in range(len(files_1)) if any(s in files_1[i] for s in years)]
        files = [os.path.join(path, f) for f in files_2 if f.endswith('.nc4') and 'tmin' in f]

    if gridded_product == "imerg" and var_name == 'precip':
        start = datetime.strptime("2001-01-01", "%Y-%m-%d")
        datelist = pd.date_range(start,end)
        years = [str(year) for year in range(start.year, end.year +1)]

        
        path = "Z:\\1.Data\\Honduras\\Clima\\Productos grillados\\IMERG"
        files_1 = os.listdir(path)
        files_2 = [files_1[i] for i in range(len(files_1)) if any(s in files_1[i] for s in years)]
        files = [os.path.join(path, f) for f in files_2 if f.endswith('.nc4')]

    if gridded_product == "mswep" and var_name == 'precip':
        path = "Z:\\1.Data\\Honduras\\Clima\\Productos grillados\\MSWEP"
        files_1 = os.listdir(path)
        files_2 = [files_1[i] for i in range(len(files_1)) if any(s in files_1[i] for s in years)]
        files = [os.path.join(path, f) for f in files_2 if f.endswith('.nc4')]

 
    data = []
    for file_path in files:
        ds= nc.Dataset(file_path)
        var = ds.variables[var_name][:]
        data.append(var)
    stacked_data = np.concatenate(data, axis=0)
    
    # Extract time and coordinate values
    times = ds.variables['time'][:]
    lats = ds.variables['latitude'][:]
    lons = ds.variables['longitude'][:]

    # Define coordinates of interest
    # coords = [(14.7413889, -87.3694444), (14.4666667, -87.0680556)]




    # Find the index of the coordinates in the netCDF file
    lat_idx = np.searchsorted(lats, [coord[0] for coord in coords])
    lon_idx = np.searchsorted(lons, [coord[1] for coord in coords])

    # Extract time series for the coordinates of interest
    time_series = pd.DataFrame(stacked_data[:, lat_idx, lon_idx], columns=[f"({lat},{lon})" for lat, lon in coords])
    time_series.columns = df_points.iloc[:,0].values
    time_series.set_index(datelist,inplace=True)
    time_series.replace(-9999.0,np.nan, inplace=True)
    
    return time_series


# AgEra5

In [10]:
df_points = pd.read_csv('Z:\\1.Data\\Honduras\\raw\\coordinates_temp.csv')
df_tmax = extract_point(gridded_product = "agera5",
              df_points = df_points,
              var_name = 'tmax')

df_tmax.to_csv("Z:\\1.Data\\Honduras\\raw\\daily\\agera5_tmax_daily.csv")
grouper = pd.Grouper(freq='M')
t_max_monthly = df_tmax.groupby(grouper).agg(lambda x: x.mean(skipna=False))
t_max_monthly.to_csv("Z:\\1.Data\\Honduras\\raw\\monthly\\agera5_tmax_monthly_mean.csv")

In [11]:
df_points = pd.read_csv('Z:\\1.Data\\Honduras\\raw\\coordinates_temp.csv')
df_tmin = extract_point(gridded_product = "agera5",
              df_points = df_points,
              var_name = 'tmin')

df_tmin.to_csv("Z:\\1.Data\\Honduras\\raw\\daily\\agera5_tmin_daily.csv")
grouper = pd.Grouper(freq='M')
t_min_monthly = df_tmin.groupby(grouper).agg(lambda x: x.mean(skipna=False))
t_min_monthly.to_csv("Z:\\1.Data\\Honduras\\raw\\monthly\\agera5_tmin_monthly_mean.csv")

In [12]:
df_points_prec = pd.read_csv('Z:\\1.Data\\Honduras\\raw\\coordinates_prec.csv')
df_prec = extract_point(gridded_product = "agera5",
              df_points = df_points_prec,
              var_name = 'precip')

df_prec.to_csv("Z:\\1.Data\\Honduras\\raw\\daily\\agera5_prec_daily.csv")
grouper = pd.Grouper(freq='M')
prec_monthly = df_prec.groupby(grouper).agg(lambda x: x.sum(skipna=False))
prec_monthly.to_csv("Z:\\1.Data\\Honduras\\raw\\monthly\\agera5_prec_monthly_sum.csv")

# Chirps Monthly

In [31]:
df_points = pd.read_csv('Z:\\1.Data\\Honduras\\raw\\coordinates_prec.csv')
path = "Z:\\1.Data\\Honduras\\Clima\\Productos grillados\\CHIRPS\\monthly\\CHIRPS_monthly_198101_202302.nc"
ds= nc.Dataset(path)
stacked_data = ds.variables['precipitation'][:]


In [39]:
st_names = df_points.iloc[:,0].values
coords = df_points.iloc[:,1:3].values.tolist()


start = datetime.strptime("1981-01-01", "%Y-%m-%d")
end = datetime.strptime("2023-02-28", "%Y-%m-%d")
datelist = pd.date_range(start,end, freq='M')

In [41]:
times = ds.variables['T'][:]
lats = ds.variables['Y'][:]
lons = ds.variables['X'][:]

# Define coordinates of interest
#coords = [(14.7413889, -87.3694444), (14.4666667, -87.0680556)]




# Find the index of the coordinates in the netCDF file
lat_idx = np.searchsorted(lats, [coord[0] for coord in coords])
lon_idx = np.searchsorted(lons, [coord[1] for coord in coords])

# Extract time series for the coordinates of interest
time_series = pd.DataFrame(stacked_data[:, lat_idx, lon_idx], columns=[f"({lat},{lon})" for lat, lon in coords])
time_series.columns = df_points.iloc[:,0].values
time_series.set_index(datelist,inplace=True)
time_series.replace(-9999.0,np.nan, inplace=True)
  

In [43]:
time_series
time_series.to_csv("Z:\\1.Data\\Honduras\\raw\\monthly\\chirps_prec_monthly_sum.csv")

# IMERG

In [28]:
import math

In [13]:
df_points_prec = pd.read_csv('Z:\\1.Data\\Honduras\\raw\\coordinates_prec.csv')
df_prec = extract_point(gridded_product = "imerg",
              df_points = df_points_prec,
              var_name = 'precip')

df_prec.to_csv("Z:\\1.Data\\Honduras\\raw\\daily\\imerg_prec_daily.csv")
grouper = pd.Grouper(freq='M')
prec_monthly = df_prec.groupby(grouper).agg(lambda x: x.sum(skipna=False))
prec_monthly.to_csv("Z:\\1.Data\\Honduras\\raw\\monthly\\imerg_prec_monthly_sum.csv")

# mswep

In [14]:
df_points_prec = pd.read_csv('Z:\\1.Data\\Honduras\\raw\\coordinates_prec.csv')
df_prec = extract_point(gridded_product = "mswep",
              df_points = df_points_prec,
              var_name = 'precip')

df_prec.to_csv("Z:\\1.Data\\Honduras\\raw\\daily\\mswep_prec_daily.csv")
grouper = pd.Grouper(freq='M')
prec_monthly = df_prec.groupby(grouper).agg(lambda x: x.sum(skipna=False))
prec_monthly.to_csv("Z:\\1.Data\\Honduras\\raw\\monthly\\mswep_prec_monthly_sum.csv")